In [ ]:

## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm
     

In [ ]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:

train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov6_training/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images
     

In [ ]:
!git clone https://github.com/meituan/YOLOv6.git ### clone the official yolov6 git repo

In [ ]:
### change the directory to yolov6
%cd YOLOv6

In [ ]:
!pwd

In [ ]:
### install all requirements
%pip install -r requirements.txt 

In [ ]:

### downloading the pretrained weight
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6s.pt

In [ ]:
%pip install addict

In [ ]:
!python tools/train.py --batch 8 --img 640 --epochs 50 --conf configs/yolov6s_finetune.py --data-path /content/drive/MyDrive/yolov6_training/dataset.yaml --device 0 --eval-interval 2 --output-dir /content/drive/MyDrive/yolov6_training/yolov6_training_results --name training_backup --save_ckpt_on_last_n_epoch 10

In [ ]:
## inference or detection on new images
!python tools/infer.py --weights /content/drive/MyDrive/yolov6_training/yolov6_training_results/training_backup8/weights/best_ckpt.pt --source /content/drive/MyDrive/yolov6_training/test_images --yaml /content/drive/MyDrive/yolov6_training/dataset.yaml --save-dir /content/drive/MyDrive/yolov6_training/test_output --device 0

In [ ]:
#display result images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/drive/MyDrive/yolov6_training/test_output/test_images/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")